# 프로덕션 환경에서 AutoGen 에이전트: 관찰 및 평가

이 튜토리얼에서는 **[Autogen 에이전트](https://github.com/microsoft/autogen)**의 내부 단계(추적)를 **모니터링**하고 [Langfuse](https://langfuse.com)를 사용하여 **성능을 평가**하는 방법을 배웁니다.

이 가이드는 팀이 에이전트를 빠르고 안정적으로 프로덕션 환경에 도입할 수 있도록 돕는 **온라인** 및 **오프라인** 평가 지표를 다룹니다.

**AI 에이전트 평가가 중요한 이유:**
- 작업이 실패하거나 최적의 결과를 생성하지 못할 때 문제를 디버깅
- 실시간으로 비용과 성능을 모니터링
- 지속적인 피드백을 통해 신뢰성과 안전성을 향상


## 1단계: 환경 변수 설정

[Langfuse Cloud](https://cloud.langfuse.com/)에 가입하거나 [Langfuse를 자체 호스팅](https://langfuse.com/self-hosting)하여 Langfuse API 키를 얻으세요.

_**참고:** 자체 호스팅 사용자는 [Terraform 모듈](https://langfuse.com/self-hosting/azure)을 사용하여 Azure에 Langfuse를 배포할 수 있습니다. 또는 [Helm 차트](https://langfuse.com/self-hosting/kubernetes-helm)를 사용하여 Kubernetes에 Langfuse를 배포할 수도 있습니다._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

환경 변수가 설정되면 이제 Langfuse 클라이언트를 초기화할 수 있습니다. `get_client()`는 환경 변수에 제공된 자격 증명을 사용하여 Langfuse 클라이언트를 초기화합니다.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## 2단계: OpenLit 계측 초기화

이제 [OpenLit](https://github.com/openlit/openlit) 계측을 초기화합니다. OpenLit은 AutoGen 작업을 자동으로 캡처하고 OpenTelemetry (OTel) 스팬을 Langfuse로 내보냅니다.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## 3단계: 에이전트 실행하기

이제 다중 턴 에이전트를 설정하여 계측을 테스트합니다.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### 추적 구조

Langfuse는 **스팬**을 포함하는 **추적**을 기록하며, 이는 에이전트의 논리 각 단계를 나타냅니다. 여기서 추적은 전체 에이전트 실행과 다음에 대한 하위 스팬을 포함합니다:
- 식사 계획 에이전트
- 영양사 에이전트

이를 통해 시간을 어디에 소비했는지, 사용된 토큰 수 등을 정확히 확인할 수 있습니다:

![Langfuse의 추적 트리](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[추적 링크](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## 온라인 평가

온라인 평가는 실제 운영 환경에서 에이전트를 평가하는 것을 의미합니다. 즉, 실제 사용 중인 프로덕션 환경에서 에이전트의 성능을 모니터링하고 결과를 지속적으로 분석하는 과정을 포함합니다.

### 프로덕션에서 추적해야 할 일반적인 지표

1. **비용** — 계측 도구를 통해 토큰 사용량을 캡처하고, 토큰당 가격을 할당하여 대략적인 비용으로 변환할 수 있습니다.
2. **지연 시간** — 각 단계 또는 전체 실행에 소요되는 시간을 관찰합니다.
3. **사용자 피드백** — 사용자가 직접 피드백(좋아요/싫어요)을 제공하여 에이전트를 개선하거나 수정하는 데 도움을 줄 수 있습니다.
4. **LLM-as-a-Judge** — 별도의 LLM을 사용하여 에이전트의 출력을 거의 실시간으로 평가합니다(예: 유해성 또는 정확성 확인).

아래는 이러한 지표의 예시를 보여줍니다.


#### 1. 비용

아래는 `gpt-4o-mini` 호출 사용량을 보여주는 스크린샷입니다. 이는 비용이 많이 드는 단계를 확인하고 에이전트를 최적화하는 데 유용합니다.

![비용](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png)

_[추적 링크](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. 지연 시간

각 단계가 완료되는 데 걸린 시간을 확인할 수 있습니다. 아래 예시에서는 전체 실행이 약 3초 정도 걸렸으며, 이를 단계별로 나눠볼 수 있습니다. 이렇게 하면 병목 현상을 파악하고 에이전트를 최적화하는 데 도움이 됩니다.

![지연 시간](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[추적 링크](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. 사용자 피드백

에이전트가 사용자 인터페이스에 통합되어 있다면, 채팅 UI에서의 좋아요/싫어요와 같은 직접적인 사용자 피드백을 기록할 수 있습니다.


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

사용자 피드백은 Langfuse에서 캡처됩니다:

![사용자 피드백은 Langfuse에서 캡처됩니다](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. 자동화된 LLM-as-a-Judge 점수 평가

LLM-as-a-Judge는 에이전트의 출력을 자동으로 평가하는 또 다른 방법입니다. 별도의 LLM 호출을 설정하여 출력의 정확성, 유해성, 스타일 또는 관심 있는 기타 기준을 평가할 수 있습니다.

**워크플로우**:
1. **평가 템플릿**을 정의합니다. 예: "텍스트가 유해한지 확인하세요."
2. `gpt-4o-mini`와 같은 judge-model을 설정합니다. 이 경우 Azure를 통해 쿼리됩니다.
3. 에이전트가 출력을 생성할 때마다 해당 출력을 템플릿과 함께 "판사" LLM에 전달합니다.
4. 판사 LLM은 평가 점수 또는 레이블을 응답하며, 이를 관찰 도구에 기록합니다.

Langfuse의 예시:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

이 예제의 답변은 "비유해적"으로 판단되었습니다.

![LLM-as-a-Judge 평가 점수](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. 관측 가능성 메트릭 개요

이 모든 메트릭은 대시보드에서 함께 시각화할 수 있습니다. 이를 통해 여러 세션에서 에이전트의 성능을 빠르게 확인할 수 있으며, 시간에 따라 품질 메트릭을 추적하는 데 도움이 됩니다.

![관측 가능성 메트릭 개요](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## 오프라인 평가

온라인 평가는 실시간 피드백을 위해 필수적이지만, **오프라인 평가**도 필요합니다. 이는 개발 전이나 개발 중에 체계적인 점검을 수행하는 것으로, 변경 사항을 프로덕션에 반영하기 전에 품질과 신뢰성을 유지하는 데 도움을 줍니다.


### 데이터셋 평가

오프라인 평가에서는 일반적으로 다음을 수행합니다:
1. 기준 데이터셋(프롬프트와 예상 출력 쌍 포함)을 준비합니다.
2. 해당 데이터셋에서 에이전트를 실행합니다.
3. 출력 결과를 예상 결과와 비교하거나 추가적인 점수 매기기 방식을 사용합니다.

아래에서는 질문과 예상 답변이 포함된 [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset)을 사용하여 이 접근 방식을 시연합니다.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

다음으로, 실행을 추적하기 위해 Langfuse에서 데이터셋 엔티티를 생성합니다. 그런 다음, 데이터셋의 각 항목을 시스템에 추가합니다.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![Langfuse의 데이터셋 항목](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### 데이터셋에서 에이전트 실행하기

먼저, Azure OpenAI 모델을 사용하여 질문에 답변하는 간단한 Autogen 에이전트를 구성합니다.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

그런 다음, 우리는 보조 함수 `my_agent()`를 정의합니다.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

마지막으로, 각 데이터셋 항목을 반복 처리하며 에이전트를 실행하고, 추적 결과를 데이터셋 항목에 연결합니다. 원하면 간단한 평가 점수를 추가할 수도 있습니다.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

다양한 에이전트 설정으로 이 과정을 반복할 수 있습니다:
- 모델 (gpt-4o-mini, gpt-4.1 등)
- 프롬프트
- 도구 (검색 사용 vs 검색 미사용)
- 에이전트 복잡도 (다중 에이전트 vs 단일 에이전트)

그런 다음 Langfuse에서 이를 나란히 비교할 수 있습니다. 이 예시에서는 에이전트를 25개의 데이터셋 질문에 대해 3번 실행했습니다. 각 실행에서는 다른 Azure OpenAI 모델을 사용했습니다. 더 큰 모델을 사용할수록 올바르게 답변한 질문의 수가 증가한다는 것을 확인할 수 있습니다 (예상대로). `correct_answer` 점수는 데이터셋에 제공된 샘플 답변을 기준으로 질문의 정확성을 판단하도록 설정된 [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals)에 의해 생성됩니다.

![데이터셋 실행 개요](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)
![데이터셋 실행 비교](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
